In [ ]:
import numpy as np
import sys
import time
import copy
import itertools
import math

#a = raw_input("masukkan jumlah node : ")
a = '40.txt'
rawmatrix= np.loadtxt (a)
matrix= np.loadtxt (a)
start_time = time.time()
permutation_paths =[]
index =[]
path=[]
load_path=[]
min_array=[]
indexmatrix =[]

def count_load(index,matrix): #hitung beban tetutup (ujung ke ujung)
	sum = 0
	for i in range (0,len(index)-1):
		sum = sum+ matrix[index[i],index[i+1]]
	sum = sum + matrix[index[-1],index[0]]
	return sum

def count_load_open(index,matrix): #hitung ga sampe ujung ke ujung
	sum = 0
	for i in range (0,len(index)-1):
		sum = sum+ matrix[index[i],index[i+1]]
	return sum

def minl(l): #nyari index dari nilai paling minal dalam array
  return l.index(min(l))

def bruteforce(matrix): #fungsi bruteforce
	for i in range(0,len(matrix)):
		index.append(i)
	permutation_paths = list(itertools.permutations(index, len(matrix)))

	i=0
	while i <= math.factorial(len(matrix)-1):
		path = permutation_paths[i]
		load_path.append(count_load(path,matrix))
		i+=1
	return permutation_paths[minl(load_path)]

def eliminasi_1elemen(matrix,x): #membuat nilai dan kolom yang sudah terpakai menjadi 99999 
	j=0
	while j<len(matrix):
		if j==x:
			matrix[:,j] = 99999
			matrix[j,:] = 99999
		j+=1
	return matrix

def eliminasi_2elemen(matrix,x,y): #membuat nilai (x,y) dan (y,x) dan kolom yang sudah terpakai menjadi 99999
	matrix[x,y]=99999
	matrix[y,x]=99999
	return matrix

def make_array(x,matrix): #mengambil baris x dari matrix untuk di jadikan 1 array baru
	temp = matrix[:,x]
	nodepath=[];
	for i in temp: 
		nodepath.append(i)
	return nodepath

def is_zero(z):
	return  z.count(0) == len(z)

def rest_array(path7): #mencari array sisa untuk di bruteforce
	fullpath=[]
	for i in range (0,len(matrix)):
		fullpath.append(i)
	return np.setdiff1d(fullpath,path7)


def zero(prevnode,path,matrix):
	j=0
	while j<len(matrix):
		if j==prevnode:
			matrix[:,j] = 0
			matrix[j,:] = 0
		j+=1

	matrix[path[-1],path[0]]=0 
	matrix[path[0],path[-1]]=0
	return matrix

def tupletoarray(x): #merubah data tipe tupple ke array
	array= []
	i=0
	while i<len(x):
		array.append(x[i])
		i=i+1

	return array
############################### cari initial node ##################

if len(matrix)<11: #check jika matrixnya apakah lebih dari 11 atau kurang, kalau kurang pakai bruteforce
	loadlist=[]
	pathlist=[]
	path = bruteforce(matrix)
	loadlist.append(count_load(path,matrix))
	pathlist.append(path)
	print (pathlist[0])
	print (loadlist[0])


else:

	def costM(matrix):
		costM=[]
		i=3
		while i<len(matrix):
			costM.append(sorted(matrix[:,i])[1]+sorted(matrix[:,i])[2])
			i+=9
		return costM

	def solve (matrix,n):

		mat = np.matrix.copy(matrix)
		center= costM(matrix)
		path.append(center.index(min(center)))
		nextarray = make_array(path[0],matrix)
		left = sorted(nextarray)[1]
		path.insert(0,nextarray.index(left))
		right = sorted(nextarray)[2]
		path.append(nextarray.index(right))
  
		prevnode=center.index(min(center))
		leftindex=path[0] 
		rightindex=path[-1]

		leftarray=make_array(leftindex,mat)
		rightarray=make_array(rightindex,mat)
		prevnode=center.index(min(center))

		count = len(mat)-n
		for x in range (1,count):
			zero(prevnode,path,mat)
			leftarray=make_array(leftindex,mat)
			rightarray=make_array(rightindex,mat)

			left = min([i for i in leftarray if i>0], default=0);
			right = min([i for i in rightarray if i>0], default=0);

			if left<right :
				prevnode=leftindex
				leftindex = leftarray.index(left)
				leftarray = make_array(leftindex,mat)
				path.insert(0,leftindex)
			else:
				prevnode=rightindex
				rightindex = rightarray.index(right)
				rightarray = make_array(rightindex,mat)
				path.append(rightindex)
    
			left = min([i for i in leftarray if i>0], default=0);
			right = min([i for i in rightarray if i>0], default=0);

		return path


	finalpath =solve(matrix,11)
	load = count_load_open(path,rawmatrix)
	rest_array = rest_array(path)
	leftindex=path[0]
	rightindex=path[-1]
	loadrest=[]
	permutation_paths_rest = list(itertools.permutations(rest_array, len(rest_array)))	
	i=0
	while i < len(permutation_paths_rest):
		temp = permutation_paths_rest[i]
		lefta = temp[-1]
		righta =temp[0]
		left = rawmatrix[leftindex,lefta]
		right = rawmatrix[rightindex,righta]
		loadrest.append(count_load_open(temp,rawmatrix)+left+right+load)
		i+=1

	minload = min(loadrest)
	temp = minl(loadrest)
	finalpath= finalpath + tupletoarray(permutation_paths_rest[temp])
	print ("path dengan path terkecil \t: " + str (finalpath))

	print ("Cost Terkecil \t\t\t: " + str (count_load(finalpath,rawmatrix)))


print("Waktu  \t\t\t: " + "%s seconds" % (time.time() - start_time))